In [2]:
def convert_to_3fps(input_video_path, output_dir):
    """동영상을 3FPS로 변환하여 저장"""
    cap = cv2.VideoCapture(input_video_path)
    input_fps = cap.get(cv2.CAP_PROP_FPS)
    output_fps = 3
    frame_skip = int(input_fps / output_fps)
    print(frame_skip)
    base_name = os.path.splitext(os.path.basename(input_video_path))[0]
    output_path = os.path.join(output_dir, f"{base_name}_3fps.mp4")

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, output_fps, (int(cap.get(3)), int(cap.get(4))))

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_idx % frame_skip == 0:
            out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()
    return output_path
import os
import cv2
input_video_path = '/home/alpaco/osh/final/final1127.mp4'
output_dir = '/home/alpaco/osh/final/converted'

os.makedirs(output_dir, exist_ok=True)

# 초당 3프레임으로 변환
converted_video_path = convert_to_3fps(input_video_path, output_dir)
print(f"초당 3프레임으로 변환된 비디오 저장 경로: {converted_video_path}")


10


KeyboardInterrupt: 

In [128]:
import cv2
import csv
import torch
import gc
from ultralytics import YOLO
import pickle

def process_video_with_dual_csv(input_video_path, output_video_path, abs_csv_path, rel_csv_path):
    # 필요한 모듈 임포트

    # 비디오 파일 열기
    cap = cv2.VideoCapture(input_video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0:
        print(f"Error: {input_video_path} 비디오 파일을 열 수 없습니다.")
        return
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # 비디오 저장 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    # 데이터 저장을 위한 딕셔너리 초기화
    data = {}

    # CSV 파일 생성 및 헤더 작성
    with open(abs_csv_path, mode='w', newline='') as abs_csv, open(rel_csv_path, mode='w', newline='') as rel_csv:
        abs_writer = csv.writer(abs_csv)
        rel_writer = csv.writer(rel_csv)

        # 헤더 작성
        header = ['frame']
        for i in range(1, 18):
            header.extend([f'x{i}', f'y{i}'])
        header.append('label')
        abs_writer.writerow(header)
        rel_writer.writerow(header)

        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # 원본 프레임 복사
            results = model.track(frame, persist=True, conf=0.1)
            existing_labels = set()

            # 현재 프레임의 데이터를 저장할 리스트 초기화
            data[frame_count] = []

            for result in results:
                if result.boxes.id is not None:
                    for box, track_id, cls_id, conf in zip(result.boxes.xyxy, result.boxes.id, result.boxes.cls, result.boxes.conf):
                        if int(cls_id) == 0:  # 사람 클래스만 필터링
                            x1, y1, x2, y2 = map(int, box)
                            label = f'ID: {track_id}'
                            if label in existing_labels:
                                continue
                            existing_labels.add(label)

                            # BBox 외부를 검정색으로 마스킹
                            masked_frame = frame.copy()
                            masked_frame[:y1, :] = 0
                            masked_frame[y2:, :] = 0
                            masked_frame[:, :x1] = 0
                            masked_frame[:, x2:] = 0

                            # 현재 BBox에 대해 YOLO Pose 모델 적용
                            keypoints_results = pose(masked_frame, imgsz=800)
                            if hasattr(keypoints_results[0], 'keypoints'):
                                keypoints = keypoints_results[0].keypoints.xy.cpu().numpy()[0]
                                if keypoints.shape[0] >= 17:
                                    # 절대 좌표로 저장 (첫 번째 CSV 파일)
                                    abs_row_data = [frame_count]
                                    # 상대 좌표로 저장 (두 번째 CSV 파일)
                                    rel_row_data = [frame_count]

                                    for point in keypoints:
                                        x_kp, y_kp = int(point[0]), int(point[1])
                                        # 절대 좌표
                                        abs_row_data.extend([x_kp, y_kp])
                                        # 상대 좌표 (BBox 기준)
                                        rel_x_kp = x_kp - x1 if x_kp > 0 else x_kp
                                        rel_y_kp = y_kp - y1 if y_kp > 0 else y_kp
                                        rel_row_data.extend([rel_x_kp, rel_y_kp])

                                    abs_row_data.append(label)
                                    rel_row_data.append(label)

                                    # CSV 파일에 작성
                                    abs_writer.writerow(abs_row_data)
                                    rel_writer.writerow(rel_row_data)

                                    # 데이터 저장
                                    detection = {
                                        'id': int(track_id),
                                        'bbox': (x1, y1, x2, y2),
                                        'keypoints': keypoints,
                                        'conf': float(conf)
                                    }
                                    data[frame_count].append(detection)

            frame_count += 1
            torch.cuda.empty_cache()
            gc.collect()

    cap.release()
    out.release()
    print(f"{output_video_path} 파일로 동영상 저장이 완료되었습니다.")
    print(f"절대 좌표 CSV: {abs_csv_path}")
    print(f"상대 좌표 CSV: {rel_csv_path}")

    # 데이터 저장
    with open('detection_data.pkl', 'wb') as f:
        pickle.dump(data, f)
    print("Detection data saved to detection_data.pkl")


# 모델 설정
model = YOLO('yolov10x.pt')
model.overrides['imgsz'] = 1920
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
pose = YOLO('yolov8l-pose.pt')
pose.to(device)

#video_path = "/home/alpaco/osh/final/converted/C_32_7_smp_su_09-11_10-41-00_a_for_DF2_3fps.mp4" # 일반인
#video_path = "/home/alpaco/osh/final/converted/Convert_PXL_20241123_090504860_3fps.mp4" # 주취자
video_path = "/home/alpaco/osh/final/converted/final1_3fps.mp4" # 혼합

#video_path = "/home/alpaco/osh/final/converted/PXL_20241123_090617530_3fps.mp4" # 음주운전
#video_path = "/home/alpaco/osh/final/converted/final1127_3fps.mp4" # 음주운전pt용

output = "/home/alpaco/osh/final/test.mp4"  
abscsv_path = "/home/alpaco/osh/final/abscsv.csv"
relcsv_path = "/home/alpaco/osh/final/relcsv.csv"


process_video_with_dual_csv(video_path,output,abscsv_path,relcsv_path)



0: 1088x1920 4 persons, 4 cars, 1 skateboard, 67.9ms
Speed: 13.3ms preprocess, 67.9ms inference, 0.7ms postprocess per image at shape (1, 3, 1088, 1920)

0: 480x800 1 person, 13.8ms
Speed: 3.5ms preprocess, 13.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 person, 14.4ms
Speed: 3.6ms preprocess, 14.4ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 2 persons, 14.1ms
Speed: 3.6ms preprocess, 14.1ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 3 persons, 14.0ms
Speed: 3.6ms preprocess, 14.0ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 800)

0: 1088x1920 4 persons, 4 cars, 60.8ms
Speed: 15.7ms preprocess, 60.8ms inference, 0.6ms postprocess per image at shape (1, 3, 1088, 1920)

0: 480x800 1 person, 16.2ms
Speed: 5.0ms preprocess, 16.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 person, 15.9ms
Speed: 4.7ms preprocess, 15.9ms inference, 2.

In [129]:
#스케일링 진행 후
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

test_data= pd.read_csv("final/relcsv.csv")

coordinate_cols = [f'x{i}' for i in range(1, 18)] + [f'y{i}' for i in range(1, 18)]
X = test_data[coordinate_cols].values  # 34개의 좌표 피처

#scaler_X = StandardScaler()
#X_normalized = scaler_X.fit_transform(X)


#test_data[coordinate_cols] = X_normalized


In [130]:
test_data.label.value_counts()

label
ID: 46.0    182
ID: 1.0      27
ID: 64.0     25
ID: 57.0     24
ID: 4.0      12
ID: 6.0      10
ID: 7.0       3
ID: 18.0      2
ID: 81.0      2
ID: 23.0      1
ID: 33.0      1
ID: 42.0      1
ID: 45.0      1
ID: 55.0      1
Name: count, dtype: int64

In [131]:
test_data

,frame,x1,y1,x2,y2,x3,y3,x4,y4,x5,...,y13,x14,y14,x15,y15,x16,y16,x17,y17,label
0,0,0,0,0,0,0,0,30,61,0,...,274,72,382,160,376,89,476,203,477,ID: 1.0
1,0,0,0,0,0,0,0,73,76,0,...,232,142,316,131,312,194,387,169,389,ID: 4.0
2,0,172,44,180,36,167,34,190,43,0,...,170,102,264,79,255,73,347,44,339,ID: 6.0
3,0,176,40,188,36,174,30,205,51,0,...,212,95,306,86,302,67,401,100,382,ID: 7.0
4,1,0,0,0,0,0,0,33,52,84,...,264,108,379,124,361,162,477,144,462,ID: 1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,198,103,79,112,65,88,68,0,0,56,...,272,109,357,99,357,82,446,126,436,ID: 46.0
288,199,83,74,97,64,74,60,120,71,0,...,264,117,367,62,358,117,467,106,421,ID: 46.0
289,200,69,78,85,67,60,64,109,72,0,...,254,141,355,51,359,150,455,37,461,ID: 46.0
290,201,56,79,72,68,46,64,99,74,0,...,269,133,371,55,391,178,433,73,475,ID: 46.0


In [132]:
columns_to_convert = test_data.columns.difference(['label'])

# float으로 변환
test_data[columns_to_convert] = test_data[columns_to_convert].astype(float)



In [133]:
test_data

,frame,x1,y1,x2,y2,x3,y3,x4,y4,x5,...,y13,x14,y14,x15,y15,x16,y16,x17,y17,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,61.0,0.0,...,274.0,72.0,382.0,160.0,376.0,89.0,476.0,203.0,477.0,ID: 1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.0,76.0,0.0,...,232.0,142.0,316.0,131.0,312.0,194.0,387.0,169.0,389.0,ID: 4.0
2,0.0,172.0,44.0,180.0,36.0,167.0,34.0,190.0,43.0,0.0,...,170.0,102.0,264.0,79.0,255.0,73.0,347.0,44.0,339.0,ID: 6.0
3,0.0,176.0,40.0,188.0,36.0,174.0,30.0,205.0,51.0,0.0,...,212.0,95.0,306.0,86.0,302.0,67.0,401.0,100.0,382.0,ID: 7.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,52.0,84.0,...,264.0,108.0,379.0,124.0,361.0,162.0,477.0,144.0,462.0,ID: 1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,198.0,103.0,79.0,112.0,65.0,88.0,68.0,0.0,0.0,56.0,...,272.0,109.0,357.0,99.0,357.0,82.0,446.0,126.0,436.0,ID: 46.0
288,199.0,83.0,74.0,97.0,64.0,74.0,60.0,120.0,71.0,0.0,...,264.0,117.0,367.0,62.0,358.0,117.0,467.0,106.0,421.0,ID: 46.0
289,200.0,69.0,78.0,85.0,67.0,60.0,64.0,109.0,72.0,0.0,...,254.0,141.0,355.0,51.0,359.0,150.0,455.0,37.0,461.0,ID: 46.0
290,201.0,56.0,79.0,72.0,68.0,46.0,64.0,99.0,74.0,0.0,...,269.0,133.0,371.0,55.0,391.0,178.0,433.0,73.0,475.0,ID: 46.0


In [134]:
test_data.groupby(['label']).size()

label
ID: 1.0      27
ID: 18.0      2
ID: 23.0      1
ID: 33.0      1
ID: 4.0      12
ID: 42.0      1
ID: 45.0      1
ID: 46.0    182
ID: 55.0      1
ID: 57.0     24
ID: 6.0      10
ID: 64.0     25
ID: 7.0       3
ID: 81.0      2
dtype: int64

In [135]:
xs, pid = [], []

for _, group in test_data.groupby(['label']):

    print(len(group))
    group = group.sort_values(by=['frame']).reset_index(drop=True)

    # 전체 프레임 생성
    all_frames = pd.DataFrame({'frame': np.arange(0, 90)})

    # 누락된 프레임을 결합
    group = all_frames.merge(group, on='frame', how='left')
    #group.interpolate(method='linear', inplace=True, axis=0) #,  limit_direction='both')
    #print(group)
    # 결측값이 여전히 남아 있다면 0으로 채우기 (필요 시)
    #print(group)
    group.fillna(0, inplace=True)
    group['label']= group['label'].astype(str)
    
    temp= group['label'].str.replace("ID: ", "").dropna().unique().tolist()
    temp.remove("0")
    # 'frame'과 'label' 제외한 데이터로 시퀀스 생성
    data_X = group.drop(columns=['frame', 'label'], errors='ignore').values
    xs.append(data_X)

    pid.append(int(float(temp[0])))
pid
    

27
2
1
1
12
1
1
182
1
24
10
25
3
2


[1, 18, 23, 33, 4, 42, 45, 46, 55, 57, 6, 64, 7, 81]

In [136]:
import numpy as np
import pandas as pd

def create_sequences(df, seq_length):
    xs, pid = [], []
    for _, group in df.groupby(['label']):
        if len(group) < 5:
            continue
        print(len(group))
        group = group.sort_values(by=['frame']).reset_index(drop=True)

        # 전체 프레임 생성
        all_frames = pd.DataFrame({'frame': np.arange(0, seq_length)})

        # 누락된 프레임을 결합
        group = all_frames.merge(group, on='frame', how='left')
        #group.interpolate(method='linear', inplace=True, axis=0) #,  limit_direction='both')
        #print(group)
        # 결측값이 여전히 남아 있다면 0으로 채우기 (필요 시)
        #print(group)
        group.fillna(0, inplace=True)
        group['label']= group['label'].astype(str)
        
        # 'frame'과 'label' 제외한 데이터로 시퀀스 생성
        temp= group['label'].str.replace("ID: ", "").dropna().unique().tolist()
        temp.remove("0")
        # 'frame'과 'label' 제외한 데이터로 시퀀스 생성
        data_X = group.drop(columns=['frame', 'label'], errors='ignore').values
        xs.append(data_X)

        pid.append(int(float(temp[0])))
    return np.array(xs), pid

sequence_length = 90


In [137]:
# 시퀀스 생성
X_seq, pid= create_sequences(test_data, sequence_length)

27
12
182
24
10
25


In [138]:
pid

[1, 4, 46, 57, 6, 64]

In [139]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from tqdm import tqdm

test_X_tensor = torch.FloatTensor(X_seq)


# 디바이스 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class BinaryLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(BinaryLSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)  # 이진 분류이므로 출력 노드를 1개로 설정

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # 마지막 시퀀스 출력 사용
        return out

# 모델 초기화
input_size = X_seq.shape[2]
hidden_size = 50
num_layers = 1

loaded_model = BinaryLSTMModel(X_seq.shape[2], 50, 1)

# map_location을 사용하여 모델 로드
loaded_model.load_state_dict(
    torch.load('/home/alpaco/project/jsw_model/90frame000_LSTM.pt', map_location=device)
)

loaded_model.to(device)
loaded_model.eval()


BinaryLSTMModel(
  (lstm): LSTM(34, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=1, bias=True)
)

In [140]:
import torch
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

correct = 0
total = 0
all_preds = []
all_labels = []
suspect_ids = []

for i in range(len(test_X_tensor)):
    data = test_X_tensor[i]
    track_id = pid[i]
    inputs = data.unsqueeze(0).to(device)
    # 모델 예측
    outputs = loaded_model(inputs)
    pred_prob = torch.sigmoid(outputs).item()
    preds = pred_prob > 0.5  # 이진 분류로 변환
    print(f'ID: {track_id}, Probability: {pred_prob}')
    if preds:
        suspect_ids.append(track_id)

print("Suspect IDs:", suspect_ids)


ID: 1, Probability: 0.0015506033087149262
ID: 4, Probability: 0.004428684711456299
ID: 46, Probability: 0.9998868703842163
ID: 57, Probability: 0.0026528199668973684
ID: 6, Probability: 0.0016981029184535146
ID: 64, Probability: 0.0019654883071780205
Suspect IDs: [46]


In [141]:
# def generate_suspect_video(input_video_path, output_video_path, detection_data_path, suspect_ids):
#     import pickle

#     # detection_data 불러오기
#     with open(detection_data_path, 'rb') as f:
#         data = pickle.load(f)

#     # 비디오 파일 열기
#     cap = cv2.VideoCapture(input_video_path)
#     fps = cap.get(cv2.CAP_PROP_FPS)
#     if fps == 0:
#         print(f"Error: {input_video_path} 비디오 파일을 열 수 없습니다.")
#         return
#     frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#     frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

#     # 비디오 저장 설정
#     fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#     out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

#     frame_count = 0

#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         detections = data.get(frame_count, [])

#         for det in detections:
#             track_id = det['id']
#             bbox = det['bbox']
#             keypoints = det['keypoints']
#             conf = det['conf']

#             x1, y1, x2, y2 = bbox
#             label = f'ID: {track_id}'

#             if track_id in suspect_ids:
#                 # 빨간색으로 bbox 그리기
#                 color = (0, 0, 255)  # Red
#             else:
#                 # 초록색으로 bbox 그리기
#                 color = (0, 255, 0)  # Green

#             # BBox 그리기
#             cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
#             cv2.putText(frame, f'{label} conf: {conf:.2f}', (x1, y1 - 10),
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

#             # 키포인트 그리기
#             for point in keypoints:
#                 x, y = int(point[0]), int(point[1])
#                 cv2.circle(frame, (x, y), radius=5, color=color, thickness=-1)

#         out.write(frame)
#         frame_count += 1

#     cap.release()
#     out.release()
#     print(f"Suspect video saved to {output_video_path}")


    
# generate_suspect_video(video_path, '/home/alpaco/osh/final/suspect_video.mp4', 'detection_data.pkl', suspect_ids)


# Car detection


In [142]:
import cv2
import torch
import pickle
import numpy as np
from ultralytics import YOLO

def process_video_with_overlay(input_video_path, output_video_path, detection_data_path, suspect_ids, vehicle_model_path):
    # 차량 및 차문 검출 모델 로드
    vehicle_model = YOLO(vehicle_model_path)
    vehicle_model.to(device)
    
    # 차량 및 차문 클래스 이름과 ID 확인
    vehicle_class_names = vehicle_model.model.names
    print("Vehicle Model Classes:", vehicle_class_names)
    # 예시: {0: 'car', 1: 'bus', 2: 'truck', 3: 'open_driver_seat'}
    
    # 주취자 검출 데이터 로드
    with open(detection_data_path, 'rb') as f:
        person_data = pickle.load(f)
    
    # 차량 바운딩 박스를 저장할 딕셔너리 초기화
    car_tracks = {}
    door_tracks = {}
    
    # 주취자가 접촉한 차량의 정보를 저장할 딕셔너리
    suspect_car_info = {}
    
    # 비디오 파일 열기
    cap = cv2.VideoCapture(input_video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0:
        print(f"Error: {input_video_path} 비디오 파일을 열 수 없습니다.")
        return
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # 비디오 저장 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 현재 프레임의 주취자 데이터 가져오기
        person_detections = person_data.get(frame_count, [])

        # 차량 및 차문 검출 및 추적
        vehicle_results = vehicle_model.track(frame, conf=0.5, persist=True, imgsz=1024)
        current_car_ids = set()
        current_car_centers = {}  # 현재 프레임의 차량 중심 좌표
        current_door_bboxes = []  # 현재 프레임의 모든 open_driver_seat 바운딩 박스 리스트 (conf >= 0.7인 것만)

        for result in vehicle_results:
            if result.boxes.id is not None:
                for box, cls_id, track_id, conf in zip(result.boxes.xyxy, result.boxes.cls, result.boxes.id, result.boxes.conf):
                    x1, y1, x2, y2 = map(int, box)
                    cls_id = int(cls_id)
                    track_id = int(track_id)
                    conf = float(conf)
                    class_name = vehicle_class_names[cls_id]
                    
                    # 바운딩 박스 그리기 및 라벨 표시
                    if cls_id == 0:  # 자동차
                        car_tracks[track_id] = {'bbox': (x1, y1, x2, y2), 'last_seen': frame_count}
                        current_car_ids.add(track_id)
                        # 현재 차량의 중심 좌표 계산
                        center_x = (x1 + x2) / 2
                        center_y = (y1 + y2) / 2
                        current_car_centers[track_id] = (center_x, center_y)
                        
                        color = (255, 0, 0)  # 파란색 - 자동차
                        # 주취자가 접촉한 차량인지 확인
                        if track_id in suspect_car_info:
                            label = f'Intoxicated person vehicle {class_name} ID:{track_id} conf:{conf:.2f}'
                        else:
                            label = f'{class_name} ID:{track_id} conf:{conf:.2f}'
                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                        cv2.putText(frame, label, (x1, y1 - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
                    elif cls_id == 3 and conf >= 0.7:  # 차문 (open_driver_seat) & conf >= 0.7
                        door_tracks[track_id] = {'bbox': (x1, y1, x2, y2), 'last_seen': frame_count}
                        current_door_bboxes.append((x1, y1, x2, y2))
                        
                        color = (0, 255, 255)  # 노란색 - open_driver_seat
                        label = f'{class_name} ID:{track_id} conf:{conf:.2f}'
                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                        cv2.putText(frame, label, (x1, y1 - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
                    else:
                        continue  # 기타 클래스는 무시하거나 conf < 0.7인 open_driver_seat 무시

        # 오버레이 상태 초기화
        overlay_color = None
        overlay_priority = 0  # 우선순위: 높을수록 높은 우선순위

        for person_det in person_detections:
            track_id = person_det['id']
            person_bbox = person_det['bbox']
            keypoints = person_det['keypoints']
            conf = person_det['conf']

            x1_p, y1_p, x2_p, y2_p = person_bbox
            label = f'ID: {track_id} conf: {conf:.2f}'

            # 주취자 여부에 따라 색상 설정
            if track_id in suspect_ids:
                color = (0, 0, 255)  # 빨간색
            else:
                color = (0, 255, 0)  # 초록색

            # 주취자 바운딩 박스 및 키포인트 그리기
            cv2.rectangle(frame, (x1_p, y1_p), (x2_p, y2_p), color, 2)
            cv2.putText(frame, label, (x1_p, y1_p - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

            for point in keypoints:
                x, y = int(point[0]), int(point[1])
                cv2.circle(frame, (x, y), radius=5, color=color, thickness=-1)

            # 주취자인 경우에만 처리
            if track_id in suspect_ids:
                # 우선순위 1: 주취자 등장 - 노란색 (이 부분은 주석 처리되어 있음)
                # if overlay_priority < 1:
                #     overlay_color = (0, 255, 255)  # 노란색 (BGR)
                #     overlay_priority = 1

                # 차량과의 겹침 확인
                for car_id, car_info in car_tracks.items():
                    car_bbox = car_info['bbox']
                    if boxes_overlap(person_bbox, car_bbox):
                        # 우선순위 2: 주취자가 차량과 겹침 - 주황색
                        if overlay_priority < 2:
                            overlay_color = (0, 165, 255)  # 주황색 (BGR)
                            overlay_priority = 2
                        # 주취자가 접촉한 차량 정보 저장
                        if car_id not in suspect_car_info:
                            # 최초 접촉 시 차량의 초기 중심 좌표 저장
                            initial_center = current_car_centers.get(car_id)
                            suspect_car_info[car_id] = {'initial_center': initial_center, 'person_bbox': person_bbox}
                            print(f"Vehicle ID {car_id} initial center recorded: {initial_center}")
                        else:
                            suspect_car_info[car_id]['person_bbox'] = person_bbox
                        break  # 겹치는 차량이 있으면 더 이상 확인하지 않음

        # 주취자가 접촉한 차량의 바운딩 박스와 현재 프레임의 모든 open_driver_seat 바운딩 박스 간의 3중 겹침 확인
        for car_id in suspect_car_info.keys():
            car_bbox = car_tracks.get(car_id, {}).get('bbox')
            if car_bbox is None:
                continue
            person_bbox = suspect_car_info[car_id].get('person_bbox')
            if person_bbox is None:
                continue
            # 우선순위 3: 주취자, 차량, 차문이 모두 겹칠 때 - 핑크색
            if overlay_priority < 3:
                for door_bbox in current_door_bboxes:
                    if boxes_overlap(car_bbox, door_bbox) and boxes_overlap(person_bbox, door_bbox):
                        overlay_color = (147, 20, 255)  # 핑크색 (BGR)
                        overlay_priority = 3
                        break
            if overlay_priority == 3:
                continue  # 핑크색 오버레이가 설정되면 다음 차량으로

            # 우선순위 4: 차량 중심 이동이 200픽셀 이상 - 빨간색
            # 현재 프레임에서 해당 차량이 검출되었는지 확인
            if car_id in current_car_centers and suspect_car_info[car_id]['initial_center'] is not None:
                current_center = current_car_centers[car_id]
                initial_center = suspect_car_info[car_id]['initial_center']
                # 이동 거리 계산
                movement = np.sqrt((current_center[0] - initial_center[0]) ** 2 + (current_center[1] - initial_center[1]) ** 2)
                print(f"Vehicle ID {car_id} movement: {movement}")
                if movement > 200:
                    if overlay_priority < 4:
                        overlay_color = (0, 0, 255)  # 빨간색 (BGR)
                        overlay_priority = 4
                    break  # 빨간색 오버레이가 설정되면 더 이상 확인하지 않음
            else:
                # 차량이 현재 프레임에서 검출되지 않으면 넘어감
                continue

        # 오버레이 적용
        if overlay_color is not None:
            overlay = frame.copy()
            color_overlay = np.full_like(frame, overlay_color, dtype=np.uint8)
            cv2.addWeighted(color_overlay, 0.3, frame, 0.7, 0, frame)

        out.write(frame)
        frame_count += 1

    cap.release()
    out.release()
    print(f"Processed video saved to {output_video_path}")

def boxes_overlap(boxA, boxB):
    # 두 바운딩 박스가 겹치는지 확인하는 함수
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    if xA < xB and yA < yB:
        return True
    else:
        return False

# 차량 및 차문 검출 모델 경로
vehicle_model_path = '/home/alpaco/hbr/YOLOv8_results/experiment241125_1024_final/weights/best.pt'

# 오버레이 적용 함수 호출
process_video_with_overlay(
    input_video_path=video_path,
    output_video_path='/home/alpaco/osh/final/overlay_video.mp4',
    detection_data_path='detection_data.pkl',
    suspect_ids=suspect_ids,
    vehicle_model_path=vehicle_model_path
)


Vehicle Model Classes: {0: 'car', 1: 'car_door_opened', 2: 'motocycle', 3: 'open_driver_seat', 4: 'person'}

0: 576x1024 2 persons, 69.3ms
Speed: 5.1ms preprocess, 69.3ms inference, 1.3ms postprocess per image at shape (1, 3, 576, 1024)



0: 576x1024 1 person, 7.5ms
Speed: 5.0ms preprocess, 7.5ms inference, 1.3ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 1 person, 7.3ms
Speed: 5.0ms preprocess, 7.3ms inference, 1.3ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 1 person, 7.3ms
Speed: 5.0ms preprocess, 7.3ms inference, 1.3ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 1 person, 7.4ms
Speed: 4.9ms preprocess, 7.4ms inference, 1.3ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 1 person, 7.4ms
Speed: 4.9ms preprocess, 7.4ms inference, 1.3ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 (no detections), 7.2ms
Speed: 4.8ms preprocess, 7.2ms inference, 0.6ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 1 person, 7.6ms
Speed: 5.0ms preprocess, 7.6ms inference, 1.3ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 (no detections), 7.3ms
Speed: 4.9ms preprocess, 7.3ms inference, 0.6ms postprocess per image at sh